In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from tqdm import tqdm

In [2]:
batch_size = 128
# epoch = 20
# tensorboard_writer = SummaryWriter()
T_0 =3
T_mult=2
eta_min = 1e-6
epsilon = 0.01
base_matrix = [[[ 2, 2 ],[2,   3 ]]]
inv_base_matrix = [[[ 1.5, -1. ],[-1.,   1. ]]]

In [3]:


class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.fc1 = nn.Linear(4,8)
        self.fc3 = nn.Linear(8, 4)
        
    def forward(self, x):
        x = x.view(-1, 4)   # reshape Variable
        x = F.relu(self.fc1(x))
#         x = F.dropout(x, 0.1)
        x = self.fc3(x)
        return x
    
model = BaseModel()
model = model.to(torch.double)
model = model.to('cuda') 
model.train()
model

BaseModel(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
)

In [4]:
class CustomDataset(Dataset):

    def __init__(self, root_dir):
        self.dataset = np.load(root_dir)
        print('number of data points', self.dataset.shape[0])

    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        x = self.dataset[idx, :,:,0]
        y = self.dataset[idx, :,:,1]
        return x,y

In [5]:
train_set = CustomDataset('train_set.npy')
val_set = CustomDataset('val_set.npy')
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=batch_size)

number of data points 1000000
number of data points 10000


In [6]:
model.load_state_dict(torch.load('small_1.pth'))
list(model.parameters())

[Parameter containing:
 tensor([[ 2.9422e-17, -1.0965e-16, -1.0821e-16, -2.9642e-16],
         [ 9.5628e-02,  2.9369e-01,  5.3154e-02, -4.4647e-01],
         [ 2.5575e-18, -5.1115e-18,  3.6952e-16,  5.6182e-16],
         [ 1.5360e+00, -1.2078e+00, -7.5969e-01,  6.3958e-01],
         [-3.6153e-01, -4.7281e-02,  4.6443e-01, -3.2432e-01],
         [ 3.5525e-01, -1.8960e-01, -6.1029e-01,  1.3732e-01],
         [-1.2802e+00,  1.0202e+00,  6.6826e-01, -5.6571e-01],
         [-6.5787e-01,  4.8745e-01,  2.9334e-01, -2.2843e-01]], device='cuda:0',
        dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor([-6.6458e-15,  7.9882e-01, -7.6495e-15, -2.4201e-01,  1.0407e+00,
          1.2244e+00,  1.9913e-01,  9.9282e-02], device='cuda:0',
        dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor([[ 1.6707e-17, -7.5081e-02,  6.4202e-17, -1.1650e+00,  5.8385e-01,
          -5.0147e-01,  1.1485e+00,  6.6387e-01],
         [ 5.9550e-19, -8.2578e-01, -8.3030e

In [7]:
W1 = model.fc1.weight.data.to('cpu').numpy()
b1 = model.fc1.bias.data.to('cpu').numpy()
W2 = model.fc3.weight.data.to('cpu').numpy()
b2 = model.fc3.bias.data.to('cpu').numpy()

In [13]:
# analyse a11
# target: 1.5- 2.25a+ 1.5b+ 1.5c- d
datapoint=1
total_error = 0
total_number = 0 
for data, target in tqdm(val_loader):
    target = target - torch.tensor(inv_base_matrix)
    target = target/epsilon
    
    data, target = Variable(data), Variable(target)
#     data = data*epsilon
    data = data.to('cuda')
#     target = target.to('cuda')
    output = model(data)
    output = output.detach().to('cpu')
#     output = output/epsilon
    total_error += torch.sum(torch.abs(output[:,:] - target.view(-1,4)[:,:]))*epsilon
    total_number += output.shape[0]*output.shape[1]

print(total_error.numpy()/total_number)

100%|██████████| 79/79 [00:00<00:00, 513.12it/s]

7.093920595790846e-05


####

model 1 test error: 7.093920595790846e-05
model 2 test error: 6.564070982629774e-05
model 3 test error: 6.788916144246224e-05

In [14]:
temp = [7.093920595790846e-05, 6.564070982629774e-05,6.788916144246224e-05]
print(np.mean(temp))
print(np.var(temp))

6.815635907555616e-05
4.7147074970144515e-12


In [27]:
# analyse a11
# target: 1.5- 2.25a+ 1.5b+ 1.5c- d
datapoint=1
total_error = 0
total_number = 0 
for data, target in tqdm(val_loader):
    target = target - torch.tensor(inv_base_matrix)
    target = target/epsilon
    
    data, target = Variable(data), Variable(target)
    
    output = torch.zeros((target.shape[0],4)).double()
    a = data.view(-1, 4)[:,0]
    b = data.view(-1, 4)[:,1]
    c = data.view(-1, 4)[:,2]
    d = data.view(-1, 4)[:,3]
    output[:,0] = -2.25*a + 1.5* b+ 1.5*c- d
    output[:,1] =  1.5*a- 1.5*b-c+d
    output[:,2] =  1.5*a- b-1.5*c+d
    output[:,3] =   - a + b+ c-  d
    data = data.to('cuda')
    neural = model(data).detach().to('cpu')
#     total_error += torch.sum(torch.abs(output[:,:] - target.view(-1,4)[:,:]))*epsilon
    total_error += torch.sum(torch.abs(output[:,:] - neural.view(-1,4)[:,:]))*epsilon
    total_number += output.shape[0]*output.shape[1]
#     print(output[:5,:])
#     print(target[:5,:])
#     break

print(total_error.numpy()/total_number)

100%|██████████| 79/79 [00:00<00:00, 322.13it/s]

0.00023209933031373393
